In [3]:
import mnist as mn

In [ ]:
import numpy as np

In [71]:
trainX=[]
for i in mn.train_images()[:50000]:
    trainX.append((i/255).reshape([784]))
trainLabels=mn.train_labels()[:50000]

In [70]:
validX=[]
for i in mn.train_images()[-10000:]:
    validX.append((i/255).reshape([784]))
validLabels=mn.train_labels()[-10000:]

In [69]:
testX=[]
for i in mn.test_images()[-10000:]:
    testX.append((i/255).reshape([784]))
testLabels=mn.test_labels()[-10000:]


In [100]:
classifiedDict=dict(zip([i for i in range(0,10)],[[] for i in range(0,10)]))
for i in range(len(trainX)):
    classifiedDict[trainLabels[i]].append(trainX[i])
    
    #if(len(classifiedDict[5])>1):
    #    break
    
    

In [102]:
classifiedDict[5][0:2]

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [103]:
pi_values=dict(zip(list(classifiedDict.keys()),[0 for i in range(0,10)]))

In [104]:
for i in classifiedDict:
    pi_values[i]=len(classifiedDict[i])/50000

In [109]:
np.cov(np.matrix([classifiedDict[0][i] for i in range(len(classifiedDict[0]))]).transpose())

(784, 784)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [196]:
meancov=dict(zip(list(classifiedDict.keys()),[[] for i in range(0,10)]))
for i in classifiedDict:
    cov=np.cov(np.matrix([classifiedDict[i][j] for j in range(len(classifiedDict[i]))]).transpose())
    meantemp=np.mean(np.matrix([classifiedDict[i][j] for j in range(len(classifiedDict[i]))]).transpose(),axis=1)
    #print(meantemp.shape)
    mean=[]
    for j in meantemp:
        mean.append(j.tolist()[0][0])
    #print(mean)
    #break
    meancov[i].append(mean)
    meancov[i].append(cov)
     




In [178]:
np.identity(5)*5

array([[ 5.,  0.,  0.,  0.,  0.],
       [ 0.,  5.,  0.,  0.,  0.],
       [ 0.,  0.,  5.,  0.,  0.],
       [ 0.,  0.,  0.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  5.]])

In [189]:
len(meancov[1][0])

784

In [167]:
from scipy.stats import multivariate_normal as mnorm

In [205]:
mnorm.logpdf(validX[0],meancov[1][0],meancov[1][1]+10000*np.identity(784))

-4330.9044026084184

In [198]:
def getP(sample,c):
    Px=[]
    for i in range(10):
        Px.append(mnorm.pdf(sample,meancov[i][0],meancov[i][1]+c*np.identity(784)))
    return Px

In [201]:
getP(validX[3],10)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [184]:
def getProb(sample,c):
    Prob=[]
    P=getP(sample,c)
    for i in range(len(P)):
        Prob.append(pi_values[i]*P[i])
    return Prob
        

In [182]:
def getLabels(validX,c):
    labels=[]
    for i in validX:
        proArray=getProb(i,c)
        labels.append(proArray.index(max(proArray)))
    return labels

In [183]:
def getAccu(validX,validLabels,c):
    preds=getLabels(validX,c)
    return(sum([1 if i==j else 0 for i,j in zip(validLabels,preds)])/len(preds))